# Testing basic tools in documentation


[x] Add    
[x] Get


In [1]:
import molsysmt as msm

In [2]:
item = msm.convert('1L2Y', 'mmcif.PdbxContainers.DataContainer')

In [3]:
item.exists('entry')

True

# entity info

In [4]:
item.getObj('entity')

DataCategory(name:'entity'
attrbuteList: ['id', 'type', 'src_method', 'pdbx_description', 'formula_weight', 'pdbx_number_of_molecules', 'pdbx_ec', 'pdbx_mutation', 'pdbx_fragment', 'details']
Data: [[1, 'polymer', 'syn', 'TC5b', 2171.413, 1, '?', '?', '?', '?']]
)

In [5]:
item.getObj('entity_poly')

DataCategory(name:'entity_poly'
attrbuteList: ['entity_id', 'type', 'nstd_linkage', 'nstd_monomer', 'pdbx_seq_one_letter_code', 'pdbx_seq_one_letter_code_can', 'pdbx_strand_id', 'pdbx_target_identifier']
Data: [[1, 'polypeptide(L)', 'no', 'no', 'NLYIQWLKDGGPSSGRPPPS', 'NLYIQWLKDGGPSSGRPPPS', 'A', '?']]
)

In [6]:
item.getObj('entity_poly_seq')

DataCategory(name:'entity_poly_seq'
attrbuteList: ['entity_id', 'num', 'mon_id', 'hetero']
Data: [[1, 1, 'ASN', 'n'], [1, 2, 'LEU', 'n'], [1, 3, 'TYR', 'n'], [1, 4, 'ILE', 'n'], [1, 5, 'GLN', 'n'], [1, 6, 'TRP', 'n'], [1, 7, 'LEU', 'n'], [1, 8, 'LYS', 'n'], [1, 9, 'ASP', 'n'], [1, 10, 'GLY', 'n'], [1, 11, 'GLY', 'n'], [1, 12, 'PRO', 'n'], [1, 13, 'SER', 'n'], [1, 14, 'SER', 'n'], [1, 15, 'GLY', 'n'], [1, 16, 'ARG', 'n'], [1, 17, 'PRO', 'n'], [1, 18, 'PRO', 'n'], [1, 19, 'PRO', 'n'], [1, 20, 'SER', 'n']]
)

In [8]:
item.getObj('pdbx_poly_seq_scheme')

DataCategory(name:'pdbx_poly_seq_scheme'
attrbuteList: ['asym_id', 'entity_id', 'seq_id', 'mon_id', 'ndb_seq_num', 'pdb_seq_num', 'auth_seq_num', 'pdb_mon_id', 'auth_mon_id', 'pdb_strand_id', 'pdb_ins_code', 'hetero']
Data: [['A', 1, 1, 'ASN', 1, 1, 1, 'ASN', 'ASN', 'A', '.', 'n'], ['A', 1, 2, 'LEU', 2, 2, 2, 'LEU', 'LEU', 'A', '.', 'n'], ['A', 1, 3, 'TYR', 3, 3, 3, 'TYR', 'TYR', 'A', '.', 'n'], ['A', 1, 4, 'ILE', 4, 4, 4, 'ILE', 'ILE', 'A', '.', 'n'], ['A', 1, 5, 'GLN', 5, 5, 5, 'GLN', 'GLN', 'A', '.', 'n'], ['A', 1, 6, 'TRP', 6, 6, 6, 'TRP', 'TRP', 'A', '.', 'n'], ['A', 1, 7, 'LEU', 7, 7, 7, 'LEU', 'LEU', 'A', '.', 'n'], ['A', 1, 8, 'LYS', 8, 8, 8, 'LYS', 'LYS', 'A', '.', 'n'], ['A', 1, 9, 'ASP', 9, 9, 9, 'ASP', 'ASP', 'A', '.', 'n'], ['A', 1, 10, 'GLY', 10, 10, 10, 'GLY', 'GLY', 'A', '.', 'n'], ['A', 1, 11, 'GLY', 11, 11, 11, 'GLY', 'GLY', 'A', '.', 'n'], ['A', 1, 12, 'PRO', 12, 12, 12, 'PRO', 'PRO', 'A', '.', 'n'], ['A', 1, 13, 'SER', 13, 13, 13, 'SER', 'SER', 'A', '.', 'n'], ['A',

In [ ]:
datacategory_atom_site = item.getObj('atom_site')

In [ ]:
aa = { jj:ii for ii,jj in enumerate(datacategory_atom_site.getAttributeList())}

In [ ]:
aa

In [ ]:
data = datacategory_atom_site.data

In [ ]:
data[1][15]

In [ ]:
for ii in item.getObj('chem_comp_atom'):
    print(ii)

In [ ]:
for ii in item.getObj('chem_comp_bond'):
    print(ii)

In [ ]:
item.getObjNameList()